<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Análisis de sentimiento
`Fecha de entrega: Noviembre 21, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

In [11]:
import pandas as pd
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
from googletrans import Translator

In [5]:
keys= pd.read_table(r"C:\Users\TOSHIBA\Downloads\2020-2\NLP\7.11.20 Clase 12\TW.txt", header=None, sep=',')
keys = dict(zip(keys[0],keys[1]))

In [6]:
#Autenticarnos
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
tweets = tweepy.Cursor(api.search,
                       q=['Huracan'],
                       lang='es',
                       geocode=f'4.60769,-74.11439,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [8]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogota')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f'https://twitter.com/i/web/status/{tweet.id}')
    
    t.append(temp)

In [9]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,@AlertaNews24 Huracán Iota en San Andréshttps:...,2020-11-16 20:59:55,Bogota,"Bogota, Colombia",1328442699360448513,https://twitter.com/i/web/status/1328442699360...
1,RT @ELTIEMPO: 🚨 Habitantes de San Andrés sosti...,2020-11-16 20:59:41,Bogota,"Valle del Cauca, Colombia",1328442642758311937,https://twitter.com/i/web/status/1328442642758...
2,@ArevaloJuanP No sr... me da pena corregirlo. ...,2020-11-16 20:59:40,Bogota,Bogota,1328442636915662848,https://twitter.com/i/web/status/1328442636915...
3,RT @ColombiaET: Ciudadanos en San Andrés y Pro...,2020-11-16 20:59:34,Bogota,ᴄᴏʟᴏᴍʙɪᴀ ||ᴊᴘ ꜰᴏʟʟows,1328442611292655617,https://twitter.com/i/web/status/1328442611292...
4,RT @ELTIEMPO: Increíble rescate de un extranje...,2020-11-16 20:59:22,Bogota,"Valle del Cauca, Colombia",1328442560205950978,https://twitter.com/i/web/status/1328442560205...


In [13]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,@AlertaNews24 Huracán Iota en San Andréshttps:...,2020-11-16 20:59:55,Bogota
1,RT @ELTIEMPO: 🚨 Habitantes de San Andrés sosti...,2020-11-16 20:59:41,Bogota
2,@ArevaloJuanP No sr... me da pena corregirlo. ...,2020-11-16 20:59:40,Bogota
3,RT @ColombiaET: Ciudadanos en San Andrés y Pro...,2020-11-16 20:59:34,Bogota
4,RT @ELTIEMPO: Increíble rescate de un extranje...,2020-11-16 20:59:22,Bogota


In [14]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [15]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

C:\Users\TOSHIBA\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,tweet,date,city,clean,links,hashtags,mentions
0,@AlertaNews24 Huracán Iota en San Andréshttps:...,2020-11-16 20:59:55,Bogota,Huracán Iota en San Andrés,[https://twitter.com/teleislas/status/13284150...,[],[@AlertaNews24]
1,RT @ELTIEMPO: 🚨 Habitantes de San Andrés sosti...,2020-11-16 20:59:41,Bogota,RT 🚨 Habitantes de San Andrés sostienen que l...,[],[],[@ELTIEMPO:]
2,@ArevaloJuanP No sr... me da pena corregirlo. ...,2020-11-16 20:59:40,Bogota,No sr... me da pena corregirlo. petro ya hubi...,[https://t.co/A7GNBrmk7h],[],[@ArevaloJuanP]
3,RT @ColombiaET: Ciudadanos en San Andrés y Pro...,2020-11-16 20:59:34,Bogota,RT Ciudadanos en San Andrés y Providencia rep...,[],[#HuracanIota],[@ColombiaET:]
4,RT @ELTIEMPO: Increíble rescate de un extranje...,2020-11-16 20:59:22,Bogota,RT Increíble rescate de un extranjero en medi...,"[https://t.co/dZs8zn4qi6, https://t.co/ztdR…]",[],[@ELTIEMPO:]


In [16]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [17]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,@AlertaNews24 Huracán Iota en San Andréshttps:...,2020-11-16 20:59:55,Bogota,Huracán Iota en San Andrés,[https://twitter.com/teleislas/status/13284150...,[],[@AlertaNews24],[]
1,RT @ELTIEMPO: 🚨 Habitantes de San Andrés sosti...,2020-11-16 20:59:41,Bogota,RT 🚨 Habitantes de San Andrés sostienen que l...,[],[],[@ELTIEMPO:],[🚨]
2,@ArevaloJuanP No sr... me da pena corregirlo. ...,2020-11-16 20:59:40,Bogota,No sr... me da pena corregirlo. petro ya hubi...,[https://t.co/A7GNBrmk7h],[],[@ArevaloJuanP],[]
3,RT @ColombiaET: Ciudadanos en San Andrés y Pro...,2020-11-16 20:59:34,Bogota,RT Ciudadanos en San Andrés y Providencia rep...,[],[#HuracanIota],[@ColombiaET:],[]
4,RT @ELTIEMPO: Increíble rescate de un extranje...,2020-11-16 20:59:22,Bogota,RT Increíble rescate de un extranjero en medi...,"[https://t.co/dZs8zn4qi6, https://t.co/ztdR…]",[],[@ELTIEMPO:],[➡️]


In [18]:
translator = Translator()
data['ing'] = data['clean'].apply(lambda valor: translator.translate(valor, src='es', dest='en'))
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis,ing
0,@AlertaNews24 Huracán Iota en San Andréshttps:...,2020-11-16 20:59:55,Bogota,Huracán Iota en San Andrés,[https://twitter.com/teleislas/status/13284150...,[],[@AlertaNews24],[],"Translated(src=es, dest=en, text=Hurricane Iot..."
1,RT @ELTIEMPO: 🚨 Habitantes de San Andrés sosti...,2020-11-16 20:59:41,Bogota,RT 🚨 Habitantes de San Andrés sostienen que l...,[],[],[@ELTIEMPO:],[🚨],"Translated(src=es, dest=en, text=RT 🚨 Inhabita..."
2,@ArevaloJuanP No sr... me da pena corregirlo. ...,2020-11-16 20:59:40,Bogota,No sr... me da pena corregirlo. petro ya hubi...,[https://t.co/A7GNBrmk7h],[],[@ArevaloJuanP],[],"Translated(src=es, dest=en, text=I don't know ..."
3,RT @ColombiaET: Ciudadanos en San Andrés y Pro...,2020-11-16 20:59:34,Bogota,RT Ciudadanos en San Andrés y Providencia rep...,[],[#HuracanIota],[@ColombiaET:],[],"Translated(src=es, dest=en, text=RT Citizens i..."
4,RT @ELTIEMPO: Increíble rescate de un extranje...,2020-11-16 20:59:22,Bogota,RT Increíble rescate de un extranjero en medi...,"[https://t.co/dZs8zn4qi6, https://t.co/ztdR…]",[],[@ELTIEMPO:],[➡️],"Translated(src=es, dest=en, text=RT Incredible..."


In [19]:
data['ingles'] = data['ing'].apply(lambda valor: valor.text)
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis,ing,ingles
0,@AlertaNews24 Huracán Iota en San Andréshttps:...,2020-11-16 20:59:55,Bogota,Huracán Iota en San Andrés,[https://twitter.com/teleislas/status/13284150...,[],[@AlertaNews24],[],"Translated(src=es, dest=en, text=Hurricane Iot...",Hurricane Iota in San Andrés
1,RT @ELTIEMPO: 🚨 Habitantes de San Andrés sosti...,2020-11-16 20:59:41,Bogota,RT 🚨 Habitantes de San Andrés sostienen que l...,[],[],[@ELTIEMPO:],[🚨],"Translated(src=es, dest=en, text=RT 🚨 Inhabita...",RT 🚨 Inhabitants of San Andrés maintain that m...
2,@ArevaloJuanP No sr... me da pena corregirlo. ...,2020-11-16 20:59:40,Bogota,No sr... me da pena corregirlo. petro ya hubi...,[https://t.co/A7GNBrmk7h],[],[@ArevaloJuanP],[],"Translated(src=es, dest=en, text=I don't know ...",I don't know ... I'm sorry to correct it. Petr...
3,RT @ColombiaET: Ciudadanos en San Andrés y Pro...,2020-11-16 20:59:34,Bogota,RT Ciudadanos en San Andrés y Providencia rep...,[],[#HuracanIota],[@ColombiaET:],[],"Translated(src=es, dest=en, text=RT Citizens i...",RT Citizens in San Andrés and Providencia repo...
4,RT @ELTIEMPO: Increíble rescate de un extranje...,2020-11-16 20:59:22,Bogota,RT Increíble rescate de un extranjero en medi...,"[https://t.co/dZs8zn4qi6, https://t.co/ztdR…]",[],[@ELTIEMPO:],[➡️],"Translated(src=es, dest=en, text=RT Incredible...",RT Incredible rescue of a foreigner in the mid...


In [20]:
data['sentimiento'] = data['ingles'].apply(lambda valor: vader.polarity_scores(valor)['compound'])
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis,ing,ingles,sentimiento
0,@AlertaNews24 Huracán Iota en San Andréshttps:...,2020-11-16 20:59:55,Bogota,Huracán Iota en San Andrés,[https://twitter.com/teleislas/status/13284150...,[],[@AlertaNews24],[],"Translated(src=es, dest=en, text=Hurricane Iot...",Hurricane Iota in San Andrés,0.0000
1,RT @ELTIEMPO: 🚨 Habitantes de San Andrés sosti...,2020-11-16 20:59:41,Bogota,RT 🚨 Habitantes de San Andrés sostienen que l...,[],[],[@ELTIEMPO:],[🚨],"Translated(src=es, dest=en, text=RT 🚨 Inhabita...",RT 🚨 Inhabitants of San Andrés maintain that m...,0.5106
2,@ArevaloJuanP No sr... me da pena corregirlo. ...,2020-11-16 20:59:40,Bogota,No sr... me da pena corregirlo. petro ya hubi...,[https://t.co/A7GNBrmk7h],[],[@ArevaloJuanP],[],"Translated(src=es, dest=en, text=I don't know ...",I don't know ... I'm sorry to correct it. Petr...,-0.0772
3,RT @ColombiaET: Ciudadanos en San Andrés y Pro...,2020-11-16 20:59:34,Bogota,RT Ciudadanos en San Andrés y Providencia rep...,[],[#HuracanIota],[@ColombiaET:],[],"Translated(src=es, dest=en, text=RT Citizens i...",RT Citizens in San Andrés and Providencia repo...,0.0000
4,RT @ELTIEMPO: Increíble rescate de un extranje...,2020-11-16 20:59:22,Bogota,RT Increíble rescate de un extranjero en medi...,"[https://t.co/dZs8zn4qi6, https://t.co/ztdR…]",[],[@ELTIEMPO:],[➡️],"Translated(src=es, dest=en, text=RT Incredible...",RT Incredible rescue of a foreigner in the mid...,0.5563


In [21]:
data[['clean', 'sentimiento']].sort_values('sentimiento')

,clean,sentimiento
81,"Está previsto, pero cuando le llega...",-0.8674
70,Este tipo se escudó en es la primera vez de ...,-0.7906
96,"RT 🚨 Con Providencia incomunicada, crece ince...",-0.5423
68,"RT 🚨 Con Providencia incomunicada, crece ince...",-0.5423
63,RT Se tiene que ser mamerto para ser tan estú...,-0.5371
...,...,...
28,RT COMUNICADO ESPECIAL N°133\n\nEL HURACÁN IO...,0.6560
11,RT ADMIRACIÓN a las Autoridades de San Andrés...,0.7668
71,RT ADMIRACIÓN a las Autoridades de San Andrés...,0.7668
6,RT ADMIRACIÓN a las Autoridades de San Andrés...,0.7668


###  `[10 pts]` Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [22]:
data.loc[90]['clean']

'   Pero primero le importa avianca y el futbol es como armero se volvio a repetir y aca tambien se sabia con antelacion habian alertas hace mas de dos dias '

In [23]:
data.loc[81]['clean']

'           Está previsto, pero cuando le llega un huracán de clase 5 en 2 días no hay plan que valga. La furia de la naturaleza es incontrolable'

###  `[15 pts]` Punto 3: ¿Cuántos tuits positivos/negativos hay en sus datos?

In [24]:
posi=data['sentimiento']>0
nega=data['sentimiento']<0

In [32]:
positivo=data.loc[posi,]
negativo=data.loc[nega,]

In [34]:
print(positivo.shape)
negativo.shape

(61, 11)


(26, 11)

###  `[BONUS 5pts]` : Diagrama de barras de la pregunta 3

###  `[BONUS 7pts]` : Nubes de palabras por sentimiento